# Homework 2: More Exploratory Data Analysis
## Gene Expression Data and Election Polls 

Due: Thursday, October 2, 2014 11:59 PM

<a href=https://raw.githubusercontent.com/cs109/2014/master/homework/HW2.ipynb download=HW2.ipynb> Download this assignment</a>

#### Submission Instructions
To submit your homework, create a folder named lastname_firstinitial_hw# and place your IPython notebooks, data files, and any other files in this folder. Your IPython Notebooks should be completely executed with the results visible in the notebook. We should not have to run any code. Compress the folder (please use .zip compression) and submit to the CS109 dropbox in the appropriate folder. If we cannot access your work because these directions are not followed correctly, we will not grade your work.


---

## Introduction

John Tukey wrote in [Exploratory Data Analysis, 1977](http://www.amazon.com/Exploratory-Data-Analysis-Wilder-Tukey/dp/0201076160/ref=pd_bbs_sr_2/103-4466654-5303007?ie=UTF8&s=books&qid=1189739816&sr=8-2): "The greatest value of a picture is when it forces us to notice what we never expected to see." In this assignment we will continue using our exploratory data analysis tools, but apply it to new sets of data: [gene expression](http://en.wikipedia.org/wiki/Gene_expression) and polls from the [2012 Presidental Election](http://en.wikipedia.org/wiki/United_States_presidential_election,_2012) and from the [2014 Senate Midterm Elections](http://en.wikipedia.org/wiki/United_States_Senate_elections,_2014).   

**First**: You will use exploratory data analysis and apply the [singular value decomposition](http://en.wikipedia.org/wiki/Singular_value_decomposition) (SVD) to a gene expression data matrix to determine if the the date that the gene expression samples are processed has large effect on the variability seen in the data.  

**Second**: You will use the polls from the 2012 Presidential Elections to determine (1) Is there a pollster bias in presidential election polls? and (2) Is the average of polls better than just one poll?

**Finally**: You will use the [HuffPost Pollster API](http://elections.huffingtonpost.com/pollster/api) to extract the polls for the current 2014 Senate Midterm Elections and provide a preliminary prediction of the result of each state.

#### Data

We will use the following data sets: 

1. A gene expression data set called `exprs_GSE5859.csv` and sample annotation table called `sampleinfo_GSE5859.csv` which are both available on Github in the 2014_data repository: [expression data set](https://github.com/cs109/2014_data/blob/master/exprs_GSE5859.csv) and [sample annotation table](https://github.com/cs109/2014_data/blob/master/sampleinfo_GSE5859.csv).  

2. Polls from the [2012 Presidential Election: Barack Obama vs Mitt Romney](http://elections.huffingtonpost.com/pollster/2012-general-election-romney-vs-obama). The polls we will use are from the [Huffington Post Pollster](http://elections.huffingtonpost.com/pollster).  

3. Polls from the [2014 Senate Midterm Elections](http://elections.huffingtonpost.com/pollster) from the [HuffPost Pollster API](http://elections.huffingtonpost.com/pollster/api). 

---

## Load Python modules

In [1]:
# special IPython command to prepare the notebook for matplotlib
%matplotlib inline 

import requests 
from StringIO import StringIO
import numpy as np
import pandas as pd # pandas
import matplotlib.pyplot as plt # module for plotting 
import datetime as dt # module for manipulating dates and times
import numpy.linalg as lin # module for performing linear algebra operations

## Problem 1

In this problem we will be using a [gene expression](http://en.wikipedia.org/wiki/Gene_expression) data set obtained from a [microarray](http://en.wikipedia.org/wiki/DNA_microarray) experiement [Read more about the specific experiment here](http://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE5859).  There are two data sets we will use:  

1. The gene expression intensities where the rows represent the features on the microarray (e.g. genes) and the columsns represent the different microarray samples.  

2. A table that contains the information about each of the samples (columns in the gene expression data set) such as the sex, the age, the treatment status, the date the samples were processed.  Each row represents one sample. 

#### Problem 1(a) 
Read in the two files from Github: [exprs_GSE5859.csv](https://github.com/cs109/2014_data/blob/master/exprs_GSE5859.csv) and [sampleinfo_GSE5859.csv](https://github.com/cs109/2014_data/blob/master/sampleinfo_GSE5859.csv) as pandas DataFrames called `exprs` and `sampleinfo`. Use the gene names as the index of the `exprs` DataFrame.

In [47]:
#your code here
#could use parse_date=<name of data> bit error prone
exprs = pd.read_csv('https://github.com/cs109/2014_data/blob/master/exprs_GSE5859.csv?raw=true',index_col=0)
sampleinfo = pd.read_csv('https://github.com/cs109/2014_data/blob/master/sampleinfo_GSE5859.csv?raw=true')


Make sure the order of the columns in the gene expression DataFrame match the order of file names in the sample annotation DataFrame. If the order of the columns the `exprs` DataFrame do not match the order of the file names in the `sampleinfo` DataFrame, reorder the columns in the `exprs` DataFrame. 

**Note**: The column names of the gene expression DataFrame are the filenames of the orignal files from which these data were obtained.  

**Hint**: The method `list.index(x)` [[read here](https://docs.python.org/2/tutorial/datastructures.html)] can be used to return the index in the list of the first item whose value is x. It is an error if there is no such item. To check if the order of the columns in `exprs` matches the order of the rows in `sampleinfo`, you can check using the method `.all()` on a Boolean or list of Booleans: 

Example code: `(exprs.columns == sampleinfo.filename).all()`

In [48]:
(exprs.columns == sampleinfo.filename).all()

False

In [63]:
#your code here
#try reindexing exprs with the files names from sampleinfo to match the order 
exprs = exprs.transpose()
exprs = exprs.sort_index()
exprs = exprs.transpose()

sampleinfo = sampleinfo.sort_values('filename')


In [64]:
(exprs.columns == sampleinfo.filename).all()

True

In [65]:
sampleinfo[exprs.columns == sampleinfo.filename]

,ethnicity,date,filename,sex
99,ASN,2005-05-13,GSM136441.CEL.gz,F
100,ASN,2005-06-10,GSM136442.CEL.gz,F
101,ASN,2005-06-10,GSM136443.CEL.gz,M
102,ASN,2005-06-10,GSM136444.CEL.gz,M
103,ASN,2005-06-10,GSM136445.CEL.gz,F
104,ASN,2005-06-10,GSM136506.CEL.gz,M
105,ASN,2005-06-23,GSM136507.CEL.gz,M
106,ASN,2005-06-23,GSM136508.CEL.gz,M
107,ASN,2005-06-23,GSM136509.CEL.gz,M
108,ASN,2005-06-23,GSM136510.CEL.gz,M


Show the head of the two tables: `exprs` and `sampleinfo`. 

In [66]:
#your code here
print exprs.head()
print sampleinfo.head()

           GSM136441.CEL.gz  GSM136442.CEL.gz  GSM136443.CEL.gz  \
1007_s_at          6.302996          6.444051          6.238090   
1053_at            7.097359          7.099125          7.308965   
117_at             5.204992          5.448902          5.440455   
121_at             7.380881          7.638094          7.720119   
1255_g_at          3.200968          3.327960          3.192055   

           GSM136444.CEL.gz  GSM136445.CEL.gz  GSM136506.CEL.gz  \
1007_s_at          6.499574          6.454842          6.171587   
1053_at            6.889842          6.880901          7.123658   
117_at             5.386826          5.147856          5.298285   
121_at             7.822919          7.914191          7.636683   
1255_g_at          3.538002          3.352793          3.207580   

           GSM136507.CEL.gz  GSM136508.CEL.gz  GSM136509.CEL.gz  \
1007_s_at          6.517539          6.543954          6.379983   
1053_at            7.689282          7.546708          7.702

#### Problem 1(b)

Extract the year and month as integers from the `sampleinfo` table. 

**Hint**: To convert a Series or a column of a pandas DataFrame that contains a date-like object, you can use the `to_datetime` function [[read here](http://pandas.pydata.org/pandas-docs/stable/timeseries.html)].  This will create a `DatetimeIndex` which can be used to extract the month and year for each row in the DataFrame. 

In [98]:
#your code here
#sampleinfo = sampleinfo.drop('year',axis=1)
sampleinfo["date"] = pd.to_datetime(sampleinfo.date)
#sampleinfo['year'] = sampleinfo.date.year
sampleinfo["year"] = map(lambda x: x.year, sampleinfo.date)
sampleinfo["month"] = map(lambda x: x.month, sampleinfo.date)
sampleinfo

,ethnicity,date,filename,sex,year,month
99,ASN,2005-05-13,GSM136441.CEL.gz,F,2005,5
100,ASN,2005-06-10,GSM136442.CEL.gz,F,2005,6
101,ASN,2005-06-10,GSM136443.CEL.gz,M,2005,6
102,ASN,2005-06-10,GSM136444.CEL.gz,M,2005,6
103,ASN,2005-06-10,GSM136445.CEL.gz,F,2005,6
104,ASN,2005-06-10,GSM136506.CEL.gz,M,2005,6
105,ASN,2005-06-23,GSM136507.CEL.gz,M,2005,6
106,ASN,2005-06-23,GSM136508.CEL.gz,M,2005,6
107,ASN,2005-06-23,GSM136509.CEL.gz,M,2005,6
108,ASN,2005-06-23,GSM136510.CEL.gz,M,2005,6


#### Problem 1(c)

Convert the dates in the `date` column from the `sampleinfo` table into days since October 31, 2002. Add a column to the `sampleinfo` DataFrame titled `elapsedInDays` containing the days since October 31, 2002.  Show the head of the `sampleinfo` DataFrame which includes the new column.  

**Hint**: Use the `datetime` module to create a new `datetime` object for the specific date October 31, 2002. Then, subtract the October 31, 2002 date from each date from the `date` column in the `sampleinfo` DataFrame. 

In [104]:
#your code here
since_date = pd.to_datetime('2002-10-31')
sampleinfo['elapsedInDays'] = map(lambda x: x.days, sampleinfo['date'] - since_date)
sampleinfo.head()

,ethnicity,date,filename,sex,year,month,elapsedInDays
99,ASN,2005-05-13,GSM136441.CEL.gz,F,2005,5,925
100,ASN,2005-06-10,GSM136442.CEL.gz,F,2005,6,953
101,ASN,2005-06-10,GSM136443.CEL.gz,M,2005,6,953
102,ASN,2005-06-10,GSM136444.CEL.gz,M,2005,6,953
103,ASN,2005-06-10,GSM136445.CEL.gz,F,2005,6,953


#### Problem 1(d)

Use exploratory analysis and the singular value decomposition (SVD) of the gene expression data matrix to determine if the date the samples were processed has large effect on the variability seen in the data or if it is just ethnicity (which is confounded with date). 

**Hint**: See the end of the [lecture from 9/23/2014 for help with SVD](http://nbviewer.ipython.org/github/cs109/2014/blob/master/lectures/lecture07/data_scraping_transcript.ipynb). 

First subset the the `sampleinfo` DataFrame to include only the CEU ethnicity.  Call this new subsetted DataFrame `sampleinfoCEU`.  Show the head of `sampleinfoCEU` DataFrame. 

In [105]:
#your code here
sampleinfoCEU = sampleinfo[sampleinfo.ethnicity == 'CEU']
sampleinfoCEU

,ethnicity,date,filename,sex,year,month,elapsedInDays
204,CEU,2005-02-16,GSM136725.CEL.gz,F,2005,2,839
205,CEU,2005-02-16,GSM136726.CEL.gz,M,2005,2,839
206,CEU,2005-06-10,GSM136727.CEL.gz,M,2005,6,953
0,CEU,2003-02-04,GSM25349.CEL.gz,M,2003,2,96
1,CEU,2003-02-04,GSM25350.CEL.gz,M,2003,2,96
2,CEU,2002-12-17,GSM25356.CEL.gz,M,2002,12,47
3,CEU,2003-01-30,GSM25357.CEL.gz,M,2003,1,91
4,CEU,2003-01-03,GSM25358.CEL.gz,M,2003,1,64
5,CEU,2003-01-16,GSM25359.CEL.gz,M,2003,1,77
6,CEU,2003-01-03,GSM25360.CEL.gz,F,2003,1,64


Next, subset the `exprs` DataFrame to only include the samples with the CEU ethnicity. Name this new subsetted DataFrame `exprsCEU`. Show the head of the `exprsCEU` DataFrame. 

In [114]:
#your code here
exprsCEU = exprs[sampleinfoCEU.filename]
exprsCEU

,GSM136725.CEL.gz,GSM136726.CEL.gz,GSM136727.CEL.gz,GSM25349.CEL.gz,GSM25350.CEL.gz,GSM25356.CEL.gz,GSM25357.CEL.gz,GSM25358.CEL.gz,GSM25359.CEL.gz,GSM25360.CEL.gz,...,GSM48655.CEL.gz,GSM48656.CEL.gz,GSM48657.CEL.gz,GSM48658.CEL.gz,GSM48660.CEL.gz,GSM48661.CEL.gz,GSM48662.CEL.gz,GSM48663.CEL.gz,GSM48664.CEL.gz,GSM48665.CEL.gz
1007_s_at,6.488579,6.809481,6.133068,6.627014,6.249807,5.934128,6.145268,6.091270,6.041186,6.050375,...,6.237984,6.704893,6.720599,6.501510,6.558100,6.618286,6.869995,5.981000,6.403285,6.248702
1053_at,7.517410,7.200596,7.280781,6.939184,6.818038,7.047962,7.422477,7.272361,7.128216,6.750719,...,7.169030,6.658916,7.280992,7.066686,7.386702,6.407958,7.089180,7.120923,7.018998,7.155419
117_at,5.247190,5.237239,5.401876,5.113570,5.074096,5.371201,5.266550,5.342047,5.063876,5.315898,...,5.095864,5.308445,5.347524,5.600514,5.232676,5.630720,4.944748,5.275577,5.770358,5.616842
121_at,7.331864,7.357102,7.607461,7.833862,7.780682,7.458197,7.655948,7.546555,7.072670,7.092984,...,7.220390,7.712825,7.671745,7.437535,7.714650,7.416252,7.746448,8.001434,7.626723,7.452299
1255_g_at,3.175846,3.105092,3.225123,3.152269,3.111747,3.018932,3.154545,3.107954,3.224284,3.114241,...,2.926689,3.103790,3.016090,3.009983,3.151203,3.199709,3.159496,3.149710,3.242780,3.433125
1294_at,7.449893,6.966849,7.018479,7.410882,7.557979,7.299338,7.085593,7.591942,7.441150,7.421111,...,7.386696,8.005122,7.628356,7.914786,7.173199,7.665905,7.563670,7.373392,7.478840,7.559692
1316_at,4.159143,4.114151,4.117132,4.297712,4.183178,4.262169,4.339762,4.122669,4.059433,4.418636,...,4.003164,4.331490,4.231121,4.118060,4.141118,4.150407,4.317541,4.285874,4.175848,4.112733
1320_at,3.685936,3.646175,3.751869,3.628234,3.632977,3.697666,3.792282,3.506593,3.468308,3.560444,...,3.696156,3.782862,3.709504,3.686914,3.725652,3.749646,3.715990,3.624856,3.552681,3.620363
1431_at,2.760064,2.760788,2.824156,2.770336,2.792908,2.613063,2.644105,2.754759,2.497562,2.829319,...,2.649474,2.937788,2.714665,2.822806,2.837083,2.954752,2.862769,2.966377,2.777286,2.661332
1438_at,5.198846,5.212348,5.187526,5.421081,5.152743,4.966595,5.314540,4.924525,5.042933,5.184165,...,5.276706,5.231541,5.540969,5.290304,5.272286,5.419614,5.097509,5.460409,4.980462,4.982561


Check to make sure the order of the columns in the `exprsCEU` DataFrame matches the rows in the `sampleinfoCEU` DataFrame.  

In [115]:
#your code here
(exprsCEU.columns == sampleinfoCEU.filename).all()

True

Compute the average gene expression intensity in the `exprsCEU` DataFrame across all the samples. For each sample in the `exprsCEU` DataFrame, subtract the average gene expression intensity from each of the samples. Show the head of the mean normalized gene expression data.  

In [126]:
#your code here
avg_g_exprs = exprsCEU.mean(axis=1)
#avg_g_exprs
normed = exprsCEU.apply(lambda x: x-avg_g_exprs, axis=0)
normed.head()

,GSM136725.CEL.gz,GSM136726.CEL.gz,GSM136727.CEL.gz,GSM25349.CEL.gz,GSM25350.CEL.gz,GSM25356.CEL.gz,GSM25357.CEL.gz,GSM25358.CEL.gz,GSM25359.CEL.gz,GSM25360.CEL.gz,...,GSM48655.CEL.gz,GSM48656.CEL.gz,GSM48657.CEL.gz,GSM48658.CEL.gz,GSM48660.CEL.gz,GSM48661.CEL.gz,GSM48662.CEL.gz,GSM48663.CEL.gz,GSM48664.CEL.gz,GSM48665.CEL.gz
1007_s_at,0.226624,0.547526,-0.128887,0.365059,-0.012149,-0.327827,-0.116687,-0.170685,-0.220769,-0.211580,...,-0.023971,0.442938,0.458643,0.239554,0.296144,0.356331,0.608040,-0.280956,0.141330,-0.013254
1053_at,0.442193,0.125379,0.205564,-0.136032,-0.257178,-0.027254,0.347260,0.197144,0.053000,-0.324497,...,0.093813,-0.416301,0.205776,-0.008531,0.311485,-0.667259,0.013964,0.045706,-0.056219,0.080203
117_at,-0.356936,-0.366887,-0.202251,-0.490556,-0.530031,-0.232926,-0.337577,-0.262080,-0.540250,-0.288228,...,-0.508262,-0.295682,-0.256603,-0.003613,-0.371451,0.026594,-0.659379,-0.328549,0.166231,0.012716
121_at,-0.083972,-0.058733,0.191626,0.418026,0.364847,0.042362,0.240113,0.130720,-0.343165,-0.322852,...,-0.195446,0.296990,0.255909,0.021700,0.298814,0.000417,0.330612,0.585598,0.210887,0.036463
1255_g_at,0.041912,-0.028842,0.091189,0.018335,-0.022187,-0.115002,0.020611,-0.025980,0.090351,-0.019693,...,-0.207245,-0.030144,-0.117844,-0.123951,0.017269,0.065775,0.025562,0.015776,0.108846,0.299192


Using this mean normalized gene expression data, compute the projection to the first Principal Component (PC1).  

**Hint**: Use the `numpy.linalg.svd()` function in the `numpy.linalg` module (or the `scipy.linalg.svd()` function in the `scipy.linalg` module) to apply an [singular value decomposition](http://en.wikipedia.org/wiki/Singular_value_decomposition) to a matrix.  

In [128]:
#your code here
#Tip: set full_matrices=False if it's sparse matrix to speed up 
u, s, v = np.linalg.svd(normed)
u.shape, s.shape, v.shape

((8793, 8793), (102,), (102, 102))

In [137]:
#Get 1st PC 
#Needed more intuition on svd funtion that resulted in u, s, v where PC1 is v.T
PC1 = v.T
PC1

array([[ -1.27439157e-01,   9.90924019e-02,  -8.81361622e-02, ...,
         -1.23520692e-02,   1.44346151e-02,   9.90147543e-02],
       [ -1.94764682e-01,   9.77550944e-02,  -1.06797188e-01, ...,
         -6.82929581e-03,   6.90918521e-03,   9.90147543e-02],
       [ -1.86414500e-01,  -1.19369341e-01,  -3.20355087e-02, ...,
          1.09586858e-02,   1.25395225e-04,   9.90147543e-02],
       ..., 
       [ -1.27138090e-01,  -2.38770319e-01,  -9.52839506e-02, ...,
          2.45435342e-03,   2.58409911e-03,   9.90147543e-02],
       [  1.21180240e-01,  -1.83565269e-01,   1.06119899e-01, ...,
          7.84648542e-03,  -9.65238582e-03,   9.90147543e-02],
       [  7.24728666e-02,  -1.32078258e-01,   9.85454813e-02, ...,
         -1.03553315e-02,   5.40063290e-03,   9.90147543e-02]])

In [140]:
PC1[:,0] #get the 1st column

array([-0.12743916, -0.19476468, -0.1864145 , -0.04120114, -0.04667916,
        0.02323035, -0.04903385,  0.02067446,  0.00849423,  0.08821779,
        0.04838172,  0.13460391, -0.03017646,  0.08654261,  0.14043381,
        0.21930879,  0.03978106,  0.11992376,  0.04268799,  0.11985249,
       -0.05684066, -0.03694419, -0.01812362,  0.04161478,  0.02348047,
       -0.05006068, -0.07296809,  0.09099799,  0.02919139,  0.08667385,
        0.0647991 ,  0.13766969,  0.07781195,  0.17033074,  0.14079489,
        0.04714323,  0.05139931, -0.009227  , -0.00585961, -0.11769737,
       -0.11599353, -0.11487241, -0.092238  , -0.09957034, -0.095896  ,
        0.0892303 ,  0.07410897,  0.10914693,  0.03136527, -0.03967716,
       -0.0501878 , -0.04329991, -0.12438774, -0.13463256, -0.12336567,
        0.07493342,  0.01826555,  0.0799834 ,  0.07549475, -0.00115702,
        0.07315554,  0.15729431,  0.05652619,  0.1644683 ,  0.05000546,
        0.21200927, -0.0613037 ,  0.19636458,  0.12149494,  0.07

Create a histogram using the values from PC1.  Use a bin size of 25.  

(array([  2.,   0.,   2.,   3.,  10.,   5.,   6.,   5.,   8.,   6.,   1.,
          3.,   2.,   8.,   7.,   3.,   9.,   6.,   3.,   3.,   4.,   2.,
          1.,   1.,   2.]),
 array([-0.19476468, -0.17820174, -0.1616388 , -0.14507587, -0.12851293,
        -0.11194999, -0.09538705, -0.07882411, -0.06226117, -0.04569823,
        -0.02913529, -0.01257235,  0.00399058,  0.02055352,  0.03711646,
         0.0536794 ,  0.07024234,  0.08680528,  0.10336822,  0.11993116,
         0.13649409,  0.15305703,  0.16961997,  0.18618291,  0.20274585,
         0.21930879]),
 <a list of 25 Patch objects>)

In [141]:
#your code here
plt.hist(PC1[:,0], bins=25)

Create a scatter plot with the days since October 31, 2002 on the x-axis and PC1 on the y-axis.

In [146]:
#your code here
plt.scatter(sampleinfoCEU.elapsedInDays, PC1[:,0])

Around what day do you notice a difference in the way the samples were processed?

(0, 200)

In [147]:
#your code here
plt.scatter(sampleinfoCEU.elapsedInDays, PC1[:,0])
plt.xlim(0,200)

Answer: A difference in the way the samples were processed is noticed around 100 days. 

## Discussion for Problem 1

*Write a brief discussion of your conclusions to the questions and tasks above in 100 words or less.*

The 1st principal component was able to indicate significant variation of gene expressions in the samples around 100 days into the experiment.


## Problem 2: Is there a pollster bias in presidential election polls?

#### Problem 2(a)

The [HuffPost Pollster](http://elections.huffingtonpost.com/pollster) contains many political polls. You can access these polls from individual races as a CSV but you can also access polls through the [HuffPost Pollster API](http://elections.huffingtonpost.com/pollster/api) to access the data.  

Read in the polls from the [2012 Presidential Election: Barack Obama vs Mitt Romney](http://elections.huffingtonpost.com/pollster/2012-general-election-romney-vs-obama) into a pandas DataFrame called `election`. For this problem, you may read in the polls for this race directly using [the CSV file](http://elections.huffingtonpost.com/pollster/2012-general-election-romney-vs-obama.csv) available from the HuffPost Pollster page.

In [149]:
#your code here
election = pd.read_csv('http://elections.huffingtonpost.com/pollster/2012-general-election-romney-vs-obama.csv')


Show the head of the `election` DataFrame. 

In [167]:
#your code here
ele = election.sort_values('Pollster')
ele.head()

,Pollster,Start Date,End Date,Entry Date/Time (ET),Number of Observations,Population,Mode,Obama,Romney,Undecided,Other,Pollster URL,Source URL,Partisan,Affiliation,Question Text,Question Iteration
404,ABC/Post,2012-02-01,2012-02-04,2012-02-06T08:52:33Z,NaN,Registered Voters,Live Phone,51.0,45.0,2.0,1.0,http://elections.huffingtonpost.com/pollster/p...,http://www.washingtonpost.com/wp-srv/politics/...,Nonpartisan,None,NaN,1
291,ABC/Post,2012-05-17,2012-05-20,2012-05-22T11:01:01Z,874.0,Registered Voters,Live Phone,49.0,46.0,2.0,3.0,http://elections.huffingtonpost.com/pollster/p...,http://www.washingtonpost.com/wp-srv/politics/...,Nonpartisan,None,NaN,1
51,ABC/Post,2012-10-20,2012-10-23,2012-10-24T17:07:36Z,1394.0,Likely Voters,Live Phone,48.0,49.0,NaN,NaN,http://elections.huffingtonpost.com/pollster/p...,http://www.washingtonpost.com/blogs/the-fix/wp...,Nonpartisan,None,NaN,1
347,ABC/Post,2012-04-05,2012-04-08,2012-04-10T11:01:56Z,NaN,Registered Voters,Live Phone,51.0,43.0,4.0,2.0,http://elections.huffingtonpost.com/pollster/p...,http://www.washingtonpost.com/wp-srv/politics/...,Nonpartisan,None,NaN,1
80,ABC/Post,2012-10-10,2012-10-13,2012-10-15T00:34:14Z,923.0,Likely Voters,Live Phone,49.0,46.0,2.0,2.0,http://elections.huffingtonpost.com/pollster/p...,http://www.langerresearch.com/uploads/1142a120...,Nonpartisan,None,NaN,1


How many polls were conducted in November? Define this number as M.  

**Hint**: Subset the `election` DataFrame for only dates in the `Start Date` column that are in November 2012.  

In [189]:
#your code here
def is_poll_conduted(strdate):
    objdate = pd.to_datetime(strdate)
    return (objdate.year == 2012 and objdate.month == 11)

ispolled = [is_poll_conduted(strdate) for strdate in election['Start Date']]
polls = election[ispolled]
polls.drop_duplicates('Pollster', inplace=True)
M= len(polls)
M

/Users/victor/anaconda3/envs/py2/lib/python2.7/site-packages/ipykernel/__main__.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


18

Answer: 18

What was the median of the number of observations in the November polls? Define this quantity as N. 

In [193]:
#your code here
N = polls['Number of Observations'].median()
N

1200.0

Answer: 1200

#### Problem 2(b)

Using the median sample size $N$ from Problem 1(a), simulate the results from a single poll:  simulate the number of votes for Obama out of a sample size $N$ where $p$ = 0.53 is the percent of voters who are voting for Obama.  

**Hint**: Use the binomial distribution with parameters $N$ and $p$ = 0.53. 

In [18]:
#your code here


Now, perform a Monte Carlo simulation to obtain the estimated percentage of Obama votes with a sample size $N$ where $N$ is the median sample size calculated in Problem 2(a). Let $p$=0.53 be the percent of voters are voting for Obama. 

**Hint**: You will repeat the simulation above 1,000 times and plot the distribution of the estimated *percent* of Obama votes from a single poll.  The results from the single poll you simulate is random variable and will be different every time you sample. 

In [19]:
#your code here

Plot the distribution of the estimated percentage of Obama votes from your single poll. What is the distribution of the estimated percentage of Obama votes? 

In [20]:
#your code here

Answer: 

What is the standard error (SE) of the estimated percentage from the poll. 

**Hint**: Remember the SE is the standard deviation (SD) of the distribution of a random variable. 

In [21]:
#your code here

#### Problem 2(c)

Now suppose we run M polls where M is the number of polls that happened in November (calculated in Problem 2(a)). Run 1,000 simulations and compute the mean of the M polls for each simulation. 

In [22]:
#your code here

What is the distribution of the average of polls?

**Hint**: Show a plot. 

In [23]:
#your code here

Answer: 

What is the standard error (SE) of the average of polls? 

In [24]:
#your code here

Answer: 

Is the SE of the average of polls larger, the same, or smaller than that the SD of a single poll (calculated in Problem 2(b))? By how much?

**Hint**: Compute a ratio of the two quantities.  

In [25]:
#your code here

Answer: 

#### Problem 2(d) 

Repeat Problem 2(c) but now record the *across poll* standard deviation in each simulation. 

In [26]:
#your code here

What is the distribution of the *across M polls* standard deviation?

**Hint**: Show a plot. 

In [27]:
#your code here

Answer: 

#### Problem 2(e) 

What is the standard deviation of M polls in our real (not simulated) 2012 presidential election data ? 

In [28]:
#your code here

Is this larger, the same, or smaller than what we expeced if polls were not biased.

In [29]:
#your code here

Answer: 

#### Problem 2(f)

**For AC209 Students**: Learn about the normal approximation for the binomial distribution and derive the results of Problem 2(b) and 2(c) analytically (using this approximation). Compare the results obtained analytically to those obtained from simulations.

In [30]:
#your code here

Answer: 

## Discussion for Problem 2

*Write a brief discussion of your conclusions to the questions and tasks above in 100 words or less.*

---


## Problem 3: Is the average of polls better than just one poll?

#### Problem 3(a)

Most undecided voters vote for one of the two candidates at the election. Therefore, the reported percentages underestimate the final value of both candidates. However, if we assume the undecided will split evenly, then the observed difference should be an unbiased estimate of the final difference. 

Add a new column to the `election` DataFrame containg the difference between Obama and Romeny called `Diff`. 

In [31]:
#your code here

#### Problem 3(b)

Make a plot of the differences for the week before the election (e.g. 5 days) where the days are on the x-axis and the differences are on the y-axis.  Add a horizontal line showing 3.9%: the difference between Obama and Romney on election day.

In [32]:
#your code here

#### Problem 3(c) 

Make a plot showing the differences by pollster where the pollsters are on the x-axis and the differences on the y-axis. 

In [33]:
#your code here

Is the *across poll* difference larger than the *between pollster* difference? 

Answer: 

#### Problem 3(d)

Take the average for each pollster and then compute the average of that. Given this difference how confident would you have been of an Obama victory?

**Hint**: Compute an estimate of the SE of this average based exclusively on the observed data. 

In [34]:
#your code here

Answer: 

#### Problem 3(e)

**For AC209 Students**: Show the difference against time and see if you can detect a trend towards the end. Use this trend to see if it improves the final estimate.

In [35]:
#your code here

Answer: 

## Discussion for Problem 3

*Write a brief discussion of your conclusions to the questions and tasks above in 100 words or less.*

---


## Problem 4

In this last problem, we will use the polls from the [2014 Senate Midterm Elections](http://elections.huffingtonpost.com/pollster) from the [HuffPost Pollster API](http://elections.huffingtonpost.com/pollster/api) to create a preliminary prediction of the result of each state. 

The HuffPost Pollster API allows you to access the data as a CSV or a JSON response by tacking ".csv" or ".json" at the end of the URLs. For example the 2012 Presidential Election could be accessed as a [.json](http://elections.huffingtonpost.com/pollster/2012-general-election-romney-vs-obama.json) instead of a [.csv](http://elections.huffingtonpost.com/pollster/2012-general-election-romney-vs-obama.csv)

#### Problem 4(a)

Read in the polls for **all** of the 2014 Senate Elections using the HuffPost API. For example, we can consider the [2014 Senate race in Kentucky between Mitch McConnell and Alison Grimes](http://elections.huffingtonpost.com/pollster/2014-kentucky-senate-mcconnell-vs-grimes). 

To search for the 2014 Senate races, use the `topics` parameter in the API [[read more about topics here](http://elections.huffingtonpost.com/pollster/api)].  

In [36]:
url_str = "http://elections.huffingtonpost.com/pollster/api/charts/?topic=2014-senate"

To list all the URLs related to the 2014 Senate races using the pollster API, we can use a list comprehension:

In [37]:
election_urls = [election['url'] + '.csv' for election in requests.get(url_str).json()]
election_urls

[u'http://elections.huffingtonpost.com/pollster/2014-kentucky-senate-mcconnell-vs-grimes.csv',
 u'http://elections.huffingtonpost.com/pollster/2014-arkansas-senate-cotton-vs-pryor.csv',
 u'http://elections.huffingtonpost.com/pollster/2014-michigan-senate-land-vs-peters.csv',
 u'http://elections.huffingtonpost.com/pollster/2014-louisiana-senate-cassidy-vs-landrieu.csv',
 u'http://elections.huffingtonpost.com/pollster/2014-new-hampshire-senate-brown-vs-shaheen.csv',
 u'http://elections.huffingtonpost.com/pollster/2014-west-virginia-senate-capito-vs-tennant.csv',
 u'http://elections.huffingtonpost.com/pollster/2014-new-hampshire-senate-bass-vs-shaheen.csv',
 u'http://elections.huffingtonpost.com/pollster/2014-north-carolina-senate-tillis-vs-hagan.csv',
 u'http://elections.huffingtonpost.com/pollster/2014-virginia-senate-gillespie-vs-warner.csv',
 u'http://elections.huffingtonpost.com/pollster/2014-colorado-senate-gardner-vs-udall.csv',
 u'http://elections.huffingtonpost.com/pollster/2014-

Because there so many Senate races, we can create a dictionary of pandas DataFrames that will be keyed by the name of the election (a string). 

In [38]:
def build_frame(url):
    """
    Returns a pandas DataFrame object containing
    the data returned from the given url
    """
    source = requests.get(url).text
    
    # Use StringIO because pd.DataFrame.from_csv requires .read() method
    s = StringIO(source)
    
    return pd.DataFrame.from_csv(s, index_col=None).convert_objects(
            convert_dates="coerce", convert_numeric=True)

In [39]:
# Makes a dictionary of pandas DataFrames keyed on election string.
dfs = dict((election.split("/")[-1][:-4], build_frame(election)) for election in election_urls)

Show the head of the DataFrame containing the polls for the 2014 Senate race in Kentucky between McConnell and Grimes.

In [40]:
#your code here

#### Problem 4(b)

For each 2014 Senate race, create a preliminary prediction of the result for that state.

In [42]:
#your code here

# Submission Instructions

To submit your homework, create a folder named **lastname_firstinitial_hw#** and place your IPython notebooks, data files, and any other files in this folder. Your IPython Notebooks should be completely executed with the results visible in the notebook. We should not have to run any code.  Compress the folder (please use .zip compression) and submit to the CS109 dropbox in the appropriate folder. *If we cannot access your work because these directions are not followed correctly, we will not grade your work.*
